In [1]:
import os
import pandas as pd
from turkish_string import title_tr

# Data Pre-Processing
We will read, pre-process and save excel files.<br>
First let's define some dictionaries to translate column names.

In [50]:
# Common function to pre-process region-base files
def process_column(col, col_dicts):
    parts = col.split(" ve ")
    # Extract multi-level columns and translete using related dictionaries
    result = []
    for i, col_dict in enumerate(col_dicts):
          result.append(col_dict.get(parts[i], parts[i] ))
    return tuple(result)
#Helper method for common modules other than names and surnames
def data_preprocess(file_path,geo_scale,col_dicts,names,need_pivot_conversion=False):
    skiprows=5 # skip 5 rows for countrywide files
    new_col_names= {'Unnamed: 0': 'year'}
    new_index = ["year"]
    place_colum_dict={"ibbs3":"province","ibbs2":"sub-region","ibbs1":"region","district":"district"}
    if "turkey" not in geo_scale:
        new_col_names["Unnamed: 1"] =  "place"
        new_index.append(place_colum_dict[geo_scale])
        skiprows=4 # skip 4 rows for regional scale files

    df= pd.read_excel(file_path,skiprows=skiprows)
    df.rename(columns= new_col_names, inplace=True)
    df["year"]=df["year"].ffill()#fillna(method='ffill')
    if df["year"].dtype== "float64":
        df["year"] = df["year"].astype(int) 

    if geo_scale != "district":# remove province codes at the end
        df[place_colum_dict[geo_scale]] = df["place"].str.split('-').str[0]
    elif geo_scale=="district": # extract province and district names from Place column, drop Place column
        df["province"] = df["place"] .str.extract(r'(\w+)\(')
        df["district"] = df["place"] .str.extract(r'\((.*?)\)')

        new_index=["year","province","district"]
    df.drop(columns=["place"],inplace=True)
    df.set_index(new_index, inplace=True)
    df = df.fillna(0)

    df.sort_index(inplace=True)
    if need_pivot_conversion:
        df=pd.pivot_table(df, values=df.columns[1], index=df.index, columns=[df.columns[0]], fill_value=0)
    df=df.astype(int)
    if len(col_dicts)>1:
        # Convert to MultiIndex
        # Apply the helper function to each column name
        tuples = [process_column(col, col_dicts) for col in df.columns]
        # Create the MultiIndex
        multi_index = pd.MultiIndex.from_tuples(tuples, names=names)
        df.columns = multi_index
    else: # not multiindex (col_dicts contains single dictionary)
        df.rename(columns=col_dicts[0], inplace=True)
     
    return df       

### 1.Population Age-Sex
### 1.1. Population Age-Sex Provinces (ibbs3)

In [51]:
dict_age = {"Bilinmeyen":"unknown"}
dict_sex = {"Erkek":"male","Kadın":"female"}

In [52]:
# We change only geo_scale as ibbs3
file_path ="data/raw/population/population-age-sex-ibbs3-2007-2024.xlsx"
df_provinces = data_preprocess(file_path,col_dicts=[dict_sex,dict_age],geo_scale="ibbs3",names=["sex","age_group"])
df_provinces.to_csv("data/preprocessed/population/age-sex-ibbs3-2007-2024.csv")
df_provinces.head(2)

sex             male                                                           \
age_group        0-4   10-14  15-19  20-24  25-29  30-34  35-39  40-44  45-49   
year province                                                                   
2007 Adana     87875  100851  96648  78022  90764  80670  70980  68068  58558   
     Adıyaman  31114   35241  32872  24563  25786  20867  17573  15887  11527   

sex                   ... female                                            \
age_group      50-54  ...  50-54  55-59    5-9  60-64  65-69  70-74  75-79   
year province         ...                                                    
2007 Adana     52112  ...  51402  39610  93870  27931  22227  16919  13775   
     Adıyaman  10253  ...  10555   9782  32072   7440   6242   4572   3973   

sex                              
age_group     80-84 85-89   90+  
year province                    
2007 Adana     7869  2810  1761  
     Adıyaman  2497  1103   656  

[2 rows x 38 columns]

In [53]:
# after pre-processing, dataframe is saved as csv files. Now let's check the dataframe
df_provinces = pd.read_csv("data/preprocessed/population/age-sex-ibbs3-2007-2024.csv",index_col=[0,1],header=[0,1])
df_provinces.index = pd.MultiIndex.from_arrays([
    df_provinces.index.get_level_values(0).astype(str),
    df_provinces.index.get_level_values(1)
], names=df_provinces.index.names)

Number of districts has changed from 923 to 973. <br>Since districts data is available since 2018 for 973 districts, we deal with it seperately.

### 1.2. Population Age-Sex district

In [5]:
import calendar
list(calendar.month_name)

['',
 'January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

In [36]:
# Unlike the first three datasets (2007-2023 period), this dataset only includes data between 2018 and 2023.
file_path = "data/raw/population/population-age-sex-district-2018-2023.xlsx"
df_districts= data_preprocess(file_path, geo_scale="district",col_dicts=[dict_sex,dict_age],names=["sex","age_group"],need_pivot_conversion=False)
df_districts.to_csv("data/preprocessed/population/age-sex-district-2018-2023.csv")
df_districts.head(2)    

sex                     male                                                  \
age_group                0-4 10-14 15-19 20-24 25-29 30-34 35-39 40-44 45-49   
year province district                                                         
2018 Adana    Aladağ     641   627   562   622   539   644   712   544   550   
              Ceyhan    6881  6963  6963  5951  5820  5836  5955  5053  4741   

sex                           ... female                                      \
age_group              50-54  ...  50-54 55-59   5-9 60-64 65-69 70-74 75-79   
year province district        ...                                              
2018 Adana    Aladağ     459  ...    448   530   628   501   350   332   229   
              Ceyhan    4784  ...   4710  4577  6580  3778  2848  2007  1483   

sex                                      
age_group              80-84 85-89  90+  
year province district                   
2018 Adana    Aladağ     187    88   36  
              Ceyhan    1025   633  268  

[2 rows x 38 columns]

In [37]:
# after pre-processing, dataframe is saved as csv files. Now let's check the dataframe
df_districts = pd.read_csv("data/preprocessed/population/age-sex-district-2018-2023.csv",index_col=[0,1,2],header=[0,1])
df_districts.head(2)

sex                     male                                                  \
age_group                0-4 10-14 15-19 20-24 25-29 30-34 35-39 40-44 45-49   
year province district                                                         
2018 Adana    Aladağ     641   627   562   622   539   644   712   544   550   
              Ceyhan    6881  6963  6963  5951  5820  5836  5955  5053  4741   

sex                           ... female                                      \
age_group              50-54  ...  50-54 55-59   5-9 60-64 65-69 70-74 75-79   
year province district        ...                                              
2018 Adana    Aladağ     459  ...    448   530   628   501   350   332   229   
              Ceyhan    4784  ...   4710  4577  6580  3778  2848  2007  1483   

sex                                      
age_group              80-84 85-89  90+  
year province district                   
2018 Adana    Aladağ     187    88   36  
              Ceyhan    1025   633  268  

[2 rows x 38 columns]

## 2.  Population Age-Sex-Maritial Status (Over age 15)
### 2.1. Population Age-Sex-Maritial Status Provinces (ibbs3)

Since 50000 rows are allowed for queries in Tuik Portal, we deal with 4 files. <br>
After preprocessing we will merge them.

In [38]:
file_folder = "data/raw/population/age-sex-marital-status"
dict_maritial_status={"Bilinmeyen":"unknown","Boşandı":"divorced","Eşi Öldü":"widowed","Hiç Evlenmedi":"never married","Evli":"married",}
names = ["sex","age_group","maritial_status"]
geo_scale="ibbs3"
df = pd.DataFrame()
for file_name in os.listdir(file_folder):
    if "ibbs3" in file_name:
        file_path = os.path.join(file_folder,file_name)
        # merge 4 dataframes (containing 4 year periods)
        df = pd.concat( [df,data_preprocess(file_path,geo_scale, [dict_sex,dict_age,dict_maritial_status],names,need_pivot_conversion=False)] )

df = df.swaplevel(1,2, axis=1) # reorder levels as sex,maritial_status,age_group
# save the merged data frame 
df.to_csv("data/preprocessed/population/age-sex-marital-status-ibbs3-2008-2023.csv")
df.head(2)

sex                male                                                 \
maritial_status unknown divorced widowed married never married unknown   
age_group         15-19    15-19   15-19   15-19         15-19   20-24   
year province                                                            
2008 Adana          0.0        2       2     420         95993     0.0   
     Adıyaman       0.0        2       0     122         32770     0.0   

sex                                                     ...  female           \
maritial_status divorced widowed married never married  ... unknown divorced   
age_group          20-24   20-24   20-24         20-24  ...   85-89    85-89   
year province                                           ...                    
2008 Adana           108       8    9225         71645  ...     0.0       54   
     Adıyaman         14       0    3439         23108  ...     0.0       13   

sex                                                                     \
maritial_status widowed married never married unknown divorced widowed   
age_group         85-89   85-89         85-89     90+      90+     90+   
year province                                                            
2008 Adana         1970     263           107     0.0       27     859   
     Adıyaman       679     164            68     0.0        7     346   

sex                                    
maritial_status married never married  
age_group           90+           90+  
year province                          
2008 Adana           73            43  
     Adıyaman        54            32  

[2 rows x 151 columns]

We can read from file as below. Note that df has three levels in columns(header=[0,1,2]).

In [39]:
pd.read_csv("data/preprocessed/population/age-sex-marital-status-ibbs3-2008-2023.csv", index_col=[0, 1], header=[0, 1, 2]).head(2)

sex                male                                                 \
maritial_status unknown divorced widowed married never married unknown   
age_group         15-19    15-19   15-19   15-19         15-19   20-24   
year province                                                            
2008 Adana          0.0        2       2     420         95993     0.0   
     Adıyaman       0.0        2       0     122         32770     0.0   

sex                                                     ...  female           \
maritial_status divorced widowed married never married  ... unknown divorced   
age_group          20-24   20-24   20-24         20-24  ...   85-89    85-89   
year province                                           ...                    
2008 Adana           108       8    9225         71645  ...     0.0       54   
     Adıyaman         14       0    3439         23108  ...     0.0       13   

sex                                                                     \
maritial_status widowed married never married unknown divorced widowed   
age_group         85-89   85-89         85-89     90+      90+     90+   
year province                                                            
2008 Adana         1970     263           107     0.0       27     859   
     Adıyaman       679     164            68     0.0        7     346   

sex                                    
maritial_status married never married  
age_group           90+           90+  
year province                          
2008 Adana           73            43  
     Adıyaman        54            32  

[2 rows x 151 columns]

### 2.2. Population Age-Sex-Maritial Status District

In [40]:
df = pd.DataFrame()
file_folder = "data/raw/population/age-sex-marital-status"
geo_scale = "district"
dict_sex = {"Erkek":"male","Kadın":"female"}
for file_name in os.listdir(file_folder):
    if "district" in file_name:
        file_path = os.path.join(file_folder,file_name)
        # merge 12 dataframes (containing 1-year periods, each year containing 3 groups of provinces)
        df = pd.concat( [df,data_preprocess(file_path,geo_scale, [dict_sex,dict_age,dict_maritial_status],names,need_pivot_conversion=False)] )

df = df.swaplevel(1,2, axis=1)
# save the merged data frame 
df.to_csv("data/preprocessed/population/age-sex-marital-status-district-2018-2023.csv")
df.head(2)

sex                        male                                         \
maritial_status        divorced married never married divorced widowed   
age_group                 15-19   15-19         15-19    20-24   20-24   
year province district                                                   
2018 Adana    Aladağ          0       3           559        1       0   
              Ceyhan          0      12          6946       20       0   

sex                                                                    ...  \
maritial_status        married never married divorced widowed married  ...   
age_group                20-24         20-24    25-29   25-29   25-29  ...   
year province district                                                 ...   
2018 Adana    Aladağ        35           586        4       0     164  ...   
              Ceyhan       566          5361       79       2    2496  ...   

sex                           female                                         \
maritial_status        never married divorced widowed married never married   
age_group                      80-84    85-89   85-89   85-89         85-89   
year province district                                                        
2018 Adana    Aladağ               5        0      67      16             5   
              Ceyhan              37        9     529      70            24   

sex                                                              male  
maritial_status        divorced widowed married never married widowed  
age_group                   90+     90+     90+           90+   15-19  
year province district                                                 
2018 Adana    Aladağ          1      31       2             2     NaN  
              Ceyhan          4     237      22             5     NaN  

[2 rows x 128 columns]

In [41]:
pd.read_csv("data/preprocessed/population/age-sex-marital-status-district-2018-2023.csv",index_col=[0,1,2],header=[0,1,2]).head(3)

sex                        male                                         \
maritial_status        divorced married never married divorced widowed   
age_group                 15-19   15-19         15-19    20-24   20-24   
year province district                                                   
2018 Adana    Aladağ          0       3           559        1       0   
              Ceyhan          0      12          6946       20       0   
              Feke            0       1           725        2       0   

sex                                                                    ...  \
maritial_status        married never married divorced widowed married  ...   
age_group                20-24         20-24    25-29   25-29   25-29  ...   
year province district                                                 ...   
2018 Adana    Aladağ        35           586        4       0     164  ...   
              Ceyhan       566          5361       79       2    2496  ...   
              Feke          42           582        7       1     148  ...   

sex                           female                                         \
maritial_status        never married divorced widowed married never married   
age_group                      80-84    85-89   85-89   85-89         85-89   
year province district                                                        
2018 Adana    Aladağ               5        0      67      16             5   
              Ceyhan              37        9     529      70            24   
              Feke                 4        1      80      21             1   

sex                                                              male  
maritial_status        divorced widowed married never married widowed  
age_group                   90+     90+     90+           90+   15-19  
year province district                                                 
2018 Adana    Aladağ          1      31       2             2     NaN  
              Ceyhan          4     237      22             5     NaN  
              Feke            1      40       3             0     NaN  

[3 rows x 128 columns]

##  3. Birth-Sex
#### 3.1. Birth-Sex ibbs3

# PROBLEM WITH ACCESSING 2024 DATA THROUGH DATABASES
Tuik database cannot be accessed anymore. So we have to parse excel file which is in another format
### UPDATING BABY NAMES DATASET WITH TOTAL NUMBER OF BIRTHS AFTER 2019


In [56]:
# Temporarily deprecated. Use the new block below
file_path = "data/raw/population/birth-sex-ibbs3-2009-2023.xlsx"
dict_sex ={"Cinsiyet:Erkek":"male","Cinsiyet:Kız":"female"}
df_birth_ibbs3 = data_preprocess(file_path, col_dicts=[dict_sex], geo_scale="ibbs3", names= ["sex"])
df_birth_ibbs3.to_csv("data/preprocessed/population/birth-sex-ibbs3-2009-2023.csv")
# after preprocessing, dataframe is saved as csv files. Now let's check the dataframe
df_birth_ibbs3.head(2)

male  female
year province               
2009 Adana     19545   18154
     Adıyaman   6808    6607

In [95]:
import pandas as pd
import os
# 1. Dosyaları yükleyelim
# file1: Mevcut 2009-2023 verileri (Header korumalı)
file1_path = 'data/preprocessed/population/birth-sex-ibbs3-2009-2023.csv'
df1 = pd.read_csv(file1_path, header=0)

# file2: Yeni verilerin olduğu Excel
file2_path = 'data/raw/population/yeni_dogum_sayilari.xls'
df2 = pd.read_excel(file2_path, sheet_name='t49', skiprows=4)
# Sütun isimlerini string yapıp temizleyelim
df2.columns = [str(c).strip() for c in df2.columns]
# İlk iki sütuna isim verelim, diğerleri yıl sütunları
cols = list(df2.columns)
cols[0], cols[1] = 'Province', 'Sex'
df2.columns = cols
# İl isimlerini aşağıya doğru doldur ve gizli boşlukları temizle
df2['Province'] = df2['Province'].ffill().astype(str).str.strip()
# Sadece cinsiyet bazlı satırları al
df2 = df2[df2['Sex'].isin(['Erkek-Boy', 'Kız-Girl'])]
# 3. İl listesini ve sırasını orijinal dosyadan alalım
ordered_provinces = df1["province"].unique().tolist()
# 4. Yılları İşleyelim (2019 - 2024)
# Sütun isminde yıl rakamını içeren herhangi bir sütunu bulur (örn: '2024.0' veya '2024(r)')
years_to_add = ['2018','2019', '2020', '2021', '2022', '2023', '2024']
results = []
# Sütunlardaki "(r)" ekini ve ".0" uzantısını siliyoruz
df2.columns = [str(c).replace('(r)', '').replace('.0', '').strip() for c in df2.columns]
for year in years_to_add:
    # O yılı içeren sütunu bul (startswith yerine 'in' kullanarak daha esnek arama yapıyoruz)
    target_col = next((c for c in df2.columns if year in c), None)
    
    if target_col is None:
        print(f"--- Uyarı: {year} yılı için sütun bulunamadı! ---")
        continue  
    print(f"+++ {year} yılı işleniyor (Bulunan Sütun: {target_col})")
    year_df = df2[['Province', 'Sex', target_col]]
    pivot_df = year_df.pivot(index='Province', columns='Sex', values=target_col)
    for full_prov_name in ordered_provinces:
        # 'Adana-1' -> 'Adana'
        clean_name = full_prov_name.rsplit('-', 1)[0].strip()
        if clean_name in pivot_df.index:
            m_val = pivot_df.loc[clean_name, 'Erkek-Boy']
            f_val = pivot_df.loc[clean_name, 'Kız-Girl']
            
            # Sayıya çevirme işlemi (Hatalı karakterleri boş geçer)
            def safe_int(v):
                try: return int(float(v))
                except: return ""

            results.append([year  , full_prov_name, safe_int(m_val), safe_int(f_val)])
# 5. Mevcut Dosyanın Altına Ekleme (Mükerrerlik Kontrolüyle)
# Eğer mevcut dosyada (file1) 2019 ve sonrası zaten varsa, onları silip güncel veriyi ekleyelim
 
# Yeni verileri ekle
new_data_df = pd.DataFrame(results)
new_data_df.columns=["year","province","male","female"]
new_data_df.to_csv("temp/new_data_df.csv",index=0)
new_data_df['year'] = pd.to_numeric(new_data_df['year']).astype(int)
# 1. df1 içindeki 2018 ve 2019 yıllarını filtreleyip çıkarıyoruz
# (Böylece bu yıllar sadece df2'den gelecek)
df1_filtered = df1.loc[(df1['year']<2018),:]
# 2. İki dataframe'i alt alta birleştiriyoruz
df_birth_ibbs3 = pd.concat([df1_filtered, new_data_df], ignore_index=True)
# 3. (Opsiyonel) Verileri yıl ve ile göre sıralayarak düzenli hale getirebilirsiniz
df_birth_ibbs3 = df_birth_ibbs3.sort_values(by=['year', 'province']).reset_index(drop=True)
# 6. Kaydetme
output_path = 'data/preprocessed/population/birth-sex-ibbs3-2009-2024.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True) 
df_birth_ibbs3.set_index(["year","province"],inplace=True)
df_birth_ibbs3.to_csv(output_path )
print(f"Dosya şuraya kaydedildi: {output_path}")
print("df_birth_ibbs3 shape:", df_birth_ibbs3.shape)

+++ 2018 yılı işleniyor (Bulunan Sütun: 2018)
+++ 2019 yılı işleniyor (Bulunan Sütun: 2019)
+++ 2020 yılı işleniyor (Bulunan Sütun: 2020)
+++ 2021 yılı işleniyor (Bulunan Sütun: 2021)
+++ 2022 yılı işleniyor (Bulunan Sütun: 2022)
+++ 2023 yılı işleniyor (Bulunan Sütun: 2023)
+++ 2024 yılı işleniyor (Bulunan Sütun: 2024)
Dosya şuraya kaydedildi: data/preprocessed/population/birth-sex-ibbs3-2009-2024.csv
df_birth.ibbs3 shape: (1296, 2)


### 3.2. Birth-Sex-district

In [57]:
file_path = "data/raw/population/birth-sex-district-2014-2023.xlsx"
df_birth_district = data_preprocess(file_path, col_dicts=[dict_sex], geo_scale="district", names=["sex"])
df_birth_district.to_csv("data/preprocessed/population/birth-district-2014-2023.csv")
# after pre-processing, dataframe is saved as csv files. Now let's check the dataframe
df_birth_district.head(2)

male  female
year province district              
2014 Adana    Aladağ     177     160
              Ceyhan    1535    1453

### 4.  Death

#### 4.1. Deaths-Sex-Month-ibbs3

In [34]:
file_path = "data/raw/population/death-sex-month-ibbs3.xlsx"
dict_sex ={"Ölenin cinsiyeti:Erkek":"male","Ölenin cinsiyeti:Kadın":"female"} 
prefix_month_tr ="Ölümün meydana geldiği ay :"
months_tr=["01. (Ocak)","02. (Şubat)","03. (Mart)","04. (Nisan)","05. (Mayıs)","06. (Haziran)","07. (Temmuz)","08. (Ağustos)","09. (Eylül)","10. (Ekim)","11. (Kasım)","12. (Aralık)"]
months_eng=["January","February","March","April","May","June","July","August","September","October","November","December"]
dict_month= {prefix_month_tr+month_tr:month_eng for (month_tr,month_eng) in zip(months_tr,months_eng)}
df_death_sex_month_ibbs3=data_preprocess(file_path,col_dicts=[dict_sex,dict_month],geo_scale="ibbs3",names=["sex","month"])
# after pre-processing, save to  as csv files and check 
df_death_sex_month_ibbs3.to_csv("data/preprocessed/population/death_sex_month_ibbs3-2009-2023.csv")
df_death_sex_month_ibbs3.head(2)

sex              male                                                       \
month         January February March April  May June July August September   
year province                                                                
2009 Adana        447      439   481   396  424  380  424    372       399   
     Adıyaman     111       91   106    91   76   87  117     90        72   

sex                    ... female                                        \
month         October  ...  March April  May June July August September   
year province          ...                                                
2009 Adana        381  ...    390   334  308  293  303    307       302   
     Adıyaman     109  ...    106    85  100   81   78     87        83   

sex                                      
month         October November December  
year province                            
2009 Adana        335      369      352  
     Adıyaman      73       86       81  

[2 rows x 24 columns]

### 5.  Most-Common-Baby-Names-ibbs3

Note: In this part we need total number of babies for normalization. So we have to use df_birth_ibbs3 dataset from previous part. 

In [96]:
import locale
locale.setlocale(locale.LC_ALL, 'tr_TR.utf8')
from turkish_string import title_tr

In [97]:
df_baby_names_male = pd.read_excel("data/raw/population/most_common_baby_names_male.xlsx",skiprows=1)
df_baby_names_male.rename(columns={"Doğum Yılı":"year","İl":"province","İsim":"name","Cinsiyet":"sex","Sayı":"count","Sıra":"rank"}, inplace=True)
df_baby_names_male[["province","name","sex"]]= df_baby_names_male[["province","name", "sex"]].map(lambda x: title_tr(x))
df_baby_names_male.set_index(["year","province"],inplace=True)
df_baby_names_male.head(2)

name    sex  count  rank
year province                               
2024 Adana     Alparslan  Erkek    206     1
     Adana         Aslan  Erkek    167     2

#### Merging  Most-Common-Baby-Names with Birth-Sex-ibbs3
Merge with the total number of births

In [98]:
df_birth_ibbs3.head(2)

male  female
year province               
2009 Adana     19545   18154
     Adıyaman   6808    6607

In [99]:
df_baby_names_male = df_baby_names_male.join(df_birth_ibbs3[["male"]],how="left")
df_baby_names_male.head(2)

name    sex  count  rank   male
year province                                      
2024 Adana     Alparslan  Erkek    206     1  13553
     Adana         Aslan  Erkek    167     2  13553

 Save the result to csv file.

In [100]:
df_baby_names_male.rename(columns={"male":"total_count"},inplace=True)
df_baby_names_male.to_csv("data/preprocessed/population/most_common_baby_names_male.csv",encoding="utf-8")
df_baby_names_male.head(2)

name    sex  count  rank  total_count
year province                                            
2024 Adana     Alparslan  Erkek    206     1        13553
     Adana         Aslan  Erkek    167     2        13553

Let's do the same merging process for female dataset.

In [101]:
df_baby_names_female = pd.read_excel("data/raw/population/most_common_baby_names_female.xlsx",skiprows=1)
df_baby_names_female.rename(columns={"Doğum Yılı":"year","İl":"province","İsim":"name","Cinsiyet":"sex","Sayı":"count","Sıra":"rank"}, inplace=True)
df_baby_names_female[["province","name","sex"]]= df_baby_names_female[["province","name", "sex"]].map(lambda x: title_tr(x))
df_baby_names_female.set_index(["year","province"],inplace=True)
df_baby_names_female.head(2)

name    sex  count  rank
year province                           
2024 Adana      Asel  Kadın    257     1
     Adana     Defne  Kadın    222     2

For female baby names, we repeat the merging(total female births) and saving the resulting dataframe to csv file.

In [102]:
df_baby_names_female = df_baby_names_female.join(df_birth_ibbs3[["female"]])
df_baby_names_female.rename(columns={"female":"total_count"},inplace=True)
df_baby_names_female.to_csv("data/preprocessed/population/most_common_baby_names_female.csv")
df_baby_names_female.head(2)

name    sex  count  rank  total_count
year province                                        
2024 Adana      Asel  Kadın    257     1        12634
     Adana     Defne  Kadın    222     2        12634

In [103]:
df_baby_names = pd.concat([df_baby_names_male, df_baby_names_female], axis=0)
df_baby_names[["sex"]] = df_baby_names[["sex"]].replace({"Erkek":"male","Kadın":"female"})
df_baby_names.to_csv("data/preprocessed/population/names_baby.csv")
df_baby_names.head()


name   sex  count  rank  total_count
year province                                           
2024 Adana     Alparslan  male    206     1        13553
     Adana         Aslan  male    167     2        13553
     Adana         Yusuf  male    155     3        13553
     Adana        Göktuğ  male    144     4        13553
     Adana     Ömer Asaf  male    126     5        13553

### 6.  Most Common Names and Surnames-ibbs3

#### 6.1  Most Common Male Names-ibbs3

In [3]:
df_names_male = pd.read_excel("data/raw/population/most_common_names_male.xlsx",skiprows=1)
df_names_male.rename(columns={"Yıl":"year","İl":"province","İsim":"name","Cinsiyet":"sex","Sayı":"count","Sıra":"rank"}, inplace=True)
df_names_male[["province","name","sex"]]= df_names_male[["province","name", "sex"]].map(lambda x: title_tr(x))
df_names_male[["sex"]]= df_names_male[["sex"]].replace(dict_sex)
df_names_male.set_index(["year","province"],inplace=True)
df_names_male.head(2)

NameError: name 'dict_sex' is not defined

In [85]:
df_age_sex = pd.read_csv("data/preprocessed/population/age-sex-ibbs3-2007-2024.csv",index_col=[0,1],header=[0,1])
df_age_sex.head(2)

sex             male                                                           \
age_group        0-4   10-14  15-19  20-24  25-29  30-34  35-39  40-44  45-49   
year province                                                                   
2007 Adana     87875  100851  96648  78022  90764  80670  70980  68068  58558   
     Adıyaman  31114   35241  32872  24563  25786  20867  17573  15887  11527   

sex                   ... female                                            \
age_group      50-54  ...  50-54  55-59    5-9  60-64  65-69  70-74  75-79   
year province         ...                                                    
2007 Adana     52112  ...  51402  39610  93870  27931  22227  16919  13775   
     Adıyaman  10253  ...  10555   9782  32072   7440   6242   4572   3973   

sex                              
age_group     80-84 85-89   90+  
year province                    
2007 Adana     7869  2810  1761  
     Adıyaman  2497  1103   656  

[2 rows x 38 columns]

Merge df_most_common_names_male with "male" column of df_age_sex.

In [86]:
df_names_male = df_names_male.join(df_age_sex["male"].sum(axis=1).to_frame("total_count"))
df_names_male.to_csv("data/preprocessed/population/most_common_names_male.csv")
df_names_male.head(2)

name    sex  count  rank  total_count
year province                                          
2024 Adana      Mehmet  Erkek  39825     1      1140836
     Adana     Mustafa  Erkek  30673     2      1140836

#### 6.2  Most Common Female Names-ibbs3

Merge df_most_common_names_female with "female" column of df_age_sex

In [87]:
df_names_female = pd.read_excel("data/raw/population/most_common_names_female.xlsx",skiprows=1,engine='openpyxl')
df_names_female.rename(columns={"Yıl":"year","İl":"province","İsim":"name","Cinsiyet":"sex","Sayı":"count","Sıra":"rank"}, inplace=True)
df_names_female[["province","name","sex"]]= df_names_female[["province","name", "sex"]].map(lambda x: title_tr(x))
df_names_female.set_index(["year","province"],inplace=True)
df_names_female.head(2)

name    sex  count  rank
year province                           
2024 Adana     Fatma  Kadın  31365     1
     Adana      Ayşe  Kadın  24843     2

Merge df_most_common_names_female with "female" column of df_age_sex

In [88]:
df_names_female = df_names_female.join(df_age_sex["female"].sum(axis=1).to_frame("total_count"))
df_names_female.to_csv("data/preprocessed/population/most_common_names_female.csv")
df_names_female.head(2)

name    sex  count  rank  total_count
year province                                        
2024 Adana     Fatma  Kadın  31365     1      1139648
     Adana      Ayşe  Kadın  24843     2      1139648

In [89]:
df_names = pd.concat([df_names_male, df_names_female], axis=0)
df_names[["sex"]] = df_names[["sex"]].replace({"Erkek":"male","Kadın":"female"})
df_names.to_csv("data/preprocessed/population/names.csv")
df_names.head()

name   sex  count  rank  total_count
year province                                         
2024 Adana      Mehmet  male  39825     1      1140836
     Adana     Mustafa  male  30673     2      1140836
     Adana       Ahmet  male  26533     3      1140836
     Adana         Ali  male  24744     4      1140836
     Adana       Yusuf  male  17266     5      1140836

#### 6.3. Most Common Surnames-ibbs3

In [90]:
df_most_common_surnames = pd.read_excel("data/raw/population/most_common_surnames.xlsx",skiprows=1)
df_most_common_surnames.rename(columns={"Yıl":"year","İl":"province","Soyİsim":"name","Sayı":"count","Sıra":"rank"}, inplace=True)
df_most_common_surnames.drop_duplicates(inplace=True)
df_most_common_surnames[["province","name"]]= df_most_common_surnames[["province","name"]].map(lambda x: title_tr(x))
df_most_common_surnames.set_index(["year","province"],inplace=True)
df_most_common_surnames.head(2)

name  count  rank
year province                     
2024 Adana     Yılmaz  26063     1
     Adana       Kaya  23669     2

In [91]:
df_most_common_surnames = df_most_common_surnames.join(df_age_sex.sum(axis=1).to_frame("total_count"))
df_most_common_surnames.to_csv("data/preprocessed/population/most_common_surnames.csv")
df_most_common_surnames.head(2)

name  count  rank  total_count
year province                                  
2024 Adana     Yılmaz  26063     1      2280484
     Adana       Kaya  23669     2      2280484

## Finally we translate codes to Turkish.

In [6]:
df = pd.read_excel("data/raw/region_codes.xlsx")

# Step 2: Replace 'alt bölgesi' with 'sub-region' in the specific column (assuming the column is named 'Region')
df['sub-region'] = df['sub-region'].str.replace('alt bölgesi', '\nsub-region')

# Step 3: Optionally, save the updated DataFrame back to the Excel file
df.to_excel('data/preprocessed/region_codes.xlsx', index=False)

# SOME TESTS

In [35]:
dict_month

{'Ölümün meydana geldiği ay :01. (Ocak)': 'January',
 'Ölümün meydana geldiği ay :02. (Şubat)': 'February',
 'Ölümün meydana geldiği ay :03. (Mart)': 'March',
 'Ölümün meydana geldiği ay :04. (Nisan)': 'April',
 'Ölümün meydana geldiği ay :05. (Mayıs)': 'May',
 'Ölümün meydana geldiği ay :06. (Haziran)': 'June',
 'Ölümün meydana geldiği ay :07. (Temmuz)': 'July',
 'Ölümün meydana geldiği ay :08. (Ağustos)': 'August',
 'Ölümün meydana geldiği ay :09. (Eylül)': 'September',
 'Ölümün meydana geldiği ay :10. (Ekim)': 'October',
 'Ölümün meydana geldiği ay :11. (Kasım)': 'November',
 'Ölümün meydana geldiği ay :12. (Aralık)': 'December'}

In [21]:
col_dicts=[dict_sex,dict_month]
geo_scale="ibbs3"
names=["sex","month"]
skiprows=5 # skip 5 rows for countrywide files
new_col_names= {'Unnamed: 0': 'year'}
new_index = ["year"]
place_colum_dict={"ibbs3":"province","ibbs2":"sub-region","ibbs1":"region","district":"district"}
if "turkey" not in geo_scale:
    new_col_names["Unnamed: 1"] =  "place"
    new_index.append(place_colum_dict[geo_scale])
    skiprows=4 # skip 4 rows for regional scale files

df= pd.read_excel(file_path,skiprows=skiprows)

df.rename(columns= new_col_names, inplace=True)
df["year"]=df["year"].ffill()#fillna(method='ffill')
if df["year"].dtype== "float64":
    df["year"] = df["year"].astype(int) 

if geo_scale != "district":# remove province codes at the end
    df[place_colum_dict[geo_scale]] = df["place"].str.split('-').str[0]
elif geo_scale=="district": # extract province and district names from Place column, drop Place column
    df["province"] = df["place"] .str.extract(r'(\w+)\(')
    df["district"] = df["place"] .str.extract(r'\((.*?)\)')

    new_index=["year","province","district"]
df.drop(columns=["place"],inplace=True)

df.set_index(new_index, inplace=True)
df = df.fillna(0)

df.sort_index(inplace=True)
 
df=df.astype(int)
if len(col_dicts)>1:
    # Convert to MultiIndex
    # Apply the helper function to each column name
    tuples = [process_column(col, col_dicts) for col in df.columns]
   # multi_index = pd.MultiIndex.from_tuples(tuples, names=names)
#df.columns = multi_index
tuples

[('Ölenin cinsiyeti:Erkek',
  ['January',
   'February',
   'March',
   'April',
   'May',
   'June',
   'July',
   'August',
   'September',
   'October',
   'November',
   'December']),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :02. (Şubat)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :03. (Mart)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :04. (Nisan)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :05. (Mayıs)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :06. (Haziran)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :07. (Temmuz)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :08. (Ağustos)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :09. (Eylül)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :10. (Ekim)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :11. (Kasım)'),
 ('Ölenin cinsiyeti:Erkek', 'Ölümün meydana geldiği ay :12. (Aralık)'),
 ('Ölenin cinsiyeti:Kadın',